In [5]:
import os
print(f"현재 PID: {os.getpid()}")

현재 PID: 4098834


In [ ]:
# ✅ 경량 UNet + MGA 버전 (병변 마스크 포함)
import os, numpy as np, torch, gc
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from glob import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from tqdm import tqdm
import cv2
from torchvision import transforms
import re

# 디바이스 설정
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 하이퍼파라미터
slice_root = "/data1/lidc-idri/slices"
bbox_csv_path = "/home/iujeong/lung_cancer/csv/allbb_noPoly.csv"
batch_size = 4
num_epochs = 10
learning_rate = 1e-4

# 채널 수 줄인 UNet
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x): return self.net(x)

class UNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super().__init__()
        filters = [32, 64, 128, 256, 512]

        self.down1 = DoubleConv(in_channels, filters[0])
        self.down2 = DoubleConv(filters[0], filters[1])
        self.down3 = DoubleConv(filters[1], filters[2])
        self.down4 = DoubleConv(filters[2], filters[3])
        self.bottom = DoubleConv(filters[3], filters[4])

        self.up4 = nn.ConvTranspose2d(filters[4], filters[3], 2, stride=2)
        self.conv4 = DoubleConv(filters[4], filters[3])
        self.up3 = nn.ConvTranspose2d(filters[3], filters[2], 2, stride=2)
        self.conv3 = DoubleConv(filters[3], filters[2])
        self.up2 = nn.ConvTranspose2d(filters[2], filters[1], 2, stride=2)
        self.conv2 = DoubleConv(filters[2], filters[1])
        self.up1 = nn.ConvTranspose2d(filters[1], filters[0], 2, stride=2)
        self.conv1 = DoubleConv(filters[1], filters[0])

        self.final = nn.Conv2d(filters[0], out_channels, 1)

    def forward(self, x):
        d1 = self.down1(x)
        d2 = self.down2(F.max_pool2d(d1, 2))
        d3 = self.down3(F.max_pool2d(d2, 2))
        d4 = self.down4(F.max_pool2d(d3, 2))
        b = self.bottom(F.max_pool2d(d4, 2))

        u4 = self.conv4(torch.cat([self.up4(b), d4], 1))
        u3 = self.conv3(torch.cat([self.up3(u4), d3], 1))
        u2 = self.conv2(torch.cat([self.up2(u3), d2], 1))
        u1 = self.conv1(torch.cat([self.up1(u2), d1], 1))

        return torch.sigmoid(self.final(u1))

# 마스크 불러오기
def load_bbox_csv(path):
    import pandas as pd
    df = pd.read_csv(path)
    bbox_dict = {}
    for _, row in df.iterrows():
        pid = row['pid']
        slice_num = int(re.findall(r'\d+', str(row['slice']))[0])
        fname = f"{pid}_slice{slice_num:03d}.npy"
        bbox = eval(row['bb'])
        bbox_dict.setdefault(fname, []).append(bbox)
    return bbox_dict

# Dataset
class CTMaskDataset(Dataset):
    def __init__(self, image_paths, mask_dict, transform=None):
        self.image_paths = image_paths
        self.mask_dict = mask_dict
        self.transform = transform

    def __getitem__(self, idx):
        path = self.image_paths[idx]
        img = np.load(path)
        img = np.clip(img, -1000, 400)
        img = (img + 1000) / 1400.
        img = cv2.resize(img, (224, 224)).astype(np.float32)

        fname = os.path.basename(path)
        mask = np.zeros((224, 224), dtype=np.float32)
        if fname in self.mask_dict:
            for x1, y1, x2, y2 in self.mask_dict[fname]:
                mask[y1:y2, x1:x2] = 1.0

        img, mask = torch.tensor(img).unsqueeze(0), torch.tensor(mask).unsqueeze(0)
        return img, mask

    def __len__(self): return len(self.image_paths)

# 학습 루프
def train():
    image_files = glob(os.path.join(slice_root, "LIDC-IDRI-*", "*.npy"))
    train_imgs, val_imgs = train_test_split(image_files, test_size=0.2, random_state=42)

    mask_dict = load_bbox_csv(bbox_csv_path)
    train_set = CTMaskDataset(train_imgs, mask_dict)
    val_set = CTMaskDataset(val_imgs, mask_dict)

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_set, batch_size=batch_size)

    model = UNet().to(device)
    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        loss_total = 0
        for imgs, masks in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            imgs, masks = imgs.to(device), masks.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, masks)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_total += loss.item()

        avg_train_loss = loss_total / len(train_loader)
        print(f"\n📚 Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}")

        # -------------------- Validation --------------------
        model.eval()
        total_loss = 0
        total_pixels = 0
        correct_pixels = 0
        ious = []

        with torch.no_grad():
            for imgs, masks in tqdm(val_loader, desc="Validation"):
                imgs, masks = imgs.to(device), masks.to(device)
                outputs = model(imgs)
                preds = (outputs > 0.5).float()

                # IoU 계산
                intersection = (preds * masks).sum(dim=(1,2,3))
                union = (preds + masks).clamp(0,1).sum(dim=(1,2,3)) - intersection
                iou = (intersection / (union + 1e-6)).mean().item()
                ious.append(iou)

                loss = criterion(outputs, masks)
                total_loss += loss.item()

                correct_pixels += (preds == masks).sum().item()
                total_pixels += torch.numel(masks)

        val_loss = total_loss / len(val_loader)
        val_acc = correct_pixels / total_pixels

        print(f"📊 Val Loss: {val_loss:.4f}, Val Accuracy: {val_acc:.4f}")
        print(f"📈 Mean IoU: {np.mean(ious):.4f}")

if __name__ == '__main__':
    train()

Using device: cuda:1


Epoch 1:   4%|▍         | 63/1570 [00:07<02:57,  8.48it/s]


KeyboardInterrupt: 

: 